In [1]:
%env CUDA_VISIBLE_DEVICES=0
from es_map import qdax_task

import numpy as np
import brax
from brax import jumpy as jp
from brax.envs import env
import jax.numpy as jnp
import jax



from brax.envs import wrappers
from brax.io import html
from brax import envs

from IPython.display import HTML, clear_output

env: CUDA_VISIBLE_DEVICES=0


In [2]:
key = jax.random.PRNGKey(777)
key_envs = jax.random.split(key, 1)

In [26]:
# To visualize a run do:
#theta,config = load_run(run_path)
#orig_env,rollout = get_rollout(theta,config)
#HTML(html.render(orig_env.sys, [s.qp for s in rollout[:50]]))

envs.create_fn("walker2d")


functools.partial(<function create at 0x7f07a59c9820>, 'walker2d')

In [4]:
def load_run(run_path):
    import json
    with open(run_path+'/config.json') as f:
        config = json.load(f)
    theta = np.load(run_path+"/theta.npy")
    return theta,config

def get_rollout(theta,config):
    import json
    with open(run_path+'/config.json') as f:
        config = json.load(f)
    theta = np.load(run_path+"/theta.npy")
    
    from es_map import jax_evaluate
    env = jax_evaluate.create_env(config["env_name"],1,1,1000)
    model = jax_evaluate.create_MLP_model(env.observation_space.shape[1],env.action_space.shape[1])
    
    key = jax.random.PRNGKey(777)
    model_params = model.init(key)
    vec,shapes,indicies = jax_evaluate.params_tree_to_vec(model_params)   # now we can do anything with vec, like get mutated copies, calculate weighted sums for ES gradient...
    # reconstructed_model_params = vec_to_params_tree(vec,shapes,indicies)
    params = jax_evaluate.vec_to_params_tree(jnp.array(theta),shapes,indicies)
    
    from brax import envs
    env_name = config["env_name"].split("_")[0]  # remove _omni from some envs
    env_fn = envs.create_fn(env_name=env_name)
    orig_env = env_fn()
    
    jit_env_reset = jax.jit(orig_env.reset)
    jit_env_step = jax.jit(orig_env.step)
    jit_inference_fn = jax.jit(model.apply)

    # do a rollout, with policy
    rollout = []
    rng = jax.random.PRNGKey(seed=0)
    state = orig_env.reset(rng=rng)
    for i in range(300):
        #print(i)

        rollout.append(state)
        act_rng, rng = jax.random.split(rng)
        act = jit_inference_fn(params, state.obs)
        state = jit_env_step(state, act)
        
    return orig_env,rollout
    

In [64]:
run_id = "mfnlsrqm"
def id_to_path(id):
    from glob import glob    
    path = glob("/scratch/ak1774/runs/large_files_jax/*"+id, recursive = False)[0]
    return path
run_path = id_to_path(run_id)

#run_path = "/scratch/ak1774/runs/large_files_jax/run-20220303_085708-pk4loduh"
import json
with open(run_path+'/config.json') as f:
    config = json.load(f)
theta = np.load(run_path+"/theta.npy")
from es_map import jax_evaluate
env = jax_evaluate.create_env(config["env_name"],1,1,1000)
model = jax_evaluate.create_MLP_model(env.observation_space.shape[1],env.action_space.shape[1])

model_params = model.init(key)
vec,shapes,indicies = jax_evaluate.params_tree_to_vec(model_params)   # now we can do anything with vec, like get mutated copies, calculate weighted sums for ES gradient...
# reconstructed_model_params = vec_to_params_tree(vec,shapes,indicies)
params = jax_evaluate.vec_to_params_tree(jnp.array(theta),shapes,indicies)

from brax import envs


env_name = config["env_name"].split("_")[0]  # remove _omni from some envs
if env_name == "walker":
    env_name = "walker2d"
env_fn = envs.create_fn(env_name=env_name)
orig_env = env_fn()

jit_env_reset = jax.jit(orig_env.reset)
jit_env_step = jax.jit(orig_env.step)
jit_inference_fn = jax.jit(model.apply)

# do a rollout, with policy
rollout = []
rng = jax.random.PRNGKey(seed=0)
state = orig_env.reset(rng=rng)
for i in range(100):
    #print(i)
    
    rollout.append(state)
    act_rng, rng = jax.random.split(rng)
    #act = jnp.array(np.tanh(np.random.randn(orig_env.action_size))) 
    act = jit_inference_fn(params, state.obs)
    act = jnp.tanh(act)
    state = jit_env_step(state, act)



In [65]:
HTML(html.render(orig_env.sys, [s.qp for s in rollout]))

DeviceArray([ 0.03663382,  0.7588672 ,  0.3316253 ,  0.8027769 ,
             -0.44279584,  0.12696712, -0.05241289,  0.3121121 ,
             -0.6790207 , -0.7828782 ,  0.9205576 ,  0.8549941 ,
              0.50791377, -0.44582707, -0.983704  ,  0.8992942 ,
             -0.70578754], dtype=float32)

In [10]:
import scipy.spatial.distance
pairwise_sq_dists = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(a, "sqeuclidean"))

In [11]:
pairwise_sq_dists

array([[ 0.        ,  1.09959494,  4.89865881,  3.70715541,  8.12663102,
         0.03658821,  0.60143969,  1.36044662,  1.89058974,  1.38921048],
       [ 1.09959494,  0.        ,  2.12180741,  1.66977026, 10.31919414,
         1.52748479,  3.20323295,  0.8218205 ,  0.11476857,  3.96140436],
       [ 4.89865881,  2.12180741,  0.        ,  0.12434685,  8.12751743,
         5.52236656,  7.4240769 ,  1.26237218,  1.46776371,  6.57851715],
       [ 3.70715541,  1.66977026,  0.12434685,  0.        ,  6.59708977,
         4.19971977,  5.73049882,  0.63054516,  1.24416403,  4.90151368],
       [ 8.12663102, 10.31919414,  8.12751743,  6.59708977,  0.        ,
         7.73251757,  6.31502343,  5.44597266, 10.86366787,  3.47756669],
       [ 0.03658821,  1.52748479,  5.52236656,  4.19971977,  7.73251757,
         0.        ,  0.34568337,  1.61551526,  2.43004004,  1.0839876 ],
       [ 0.60143969,  3.20323295,  7.4240769 ,  5.73049882,  6.31502343,
         0.34568337,  0.        ,  2.56587161

In [6]:
from es_map import jax_es_update
import time

In [4]:
run_path = "/scratch/ak1774/runs/large_files_jax/run-20220303_085708-pk4loduh"
import json
with open(run_path+'/config.json') as f:
    config = json.load(f)
config["ENTROPY_CALCULATION_KERNEL_BANDWIDTH"] = 0.25

In [8]:
perturbations = jnp.array(np.random.randn(10000,60000))
child_fitness = jnp.array(np.random.randn(10000))
bds = jnp.array(np.random.randn(10000,2))
novelties = jnp.array(np.random.randn(10000))

modes = [
    "fitness",
    "evo_var",
    "evo_ent",
    "innovation",
    "quality_evo_var",
    "quality_evo_ent",
    "quality_innovation",
    "quality_evo_var_innovation",
    "quality_evo_ent_innovation",
]
for mode in modes:
    
    now = time.time()
    grad = jax_es_update.jax_calculate_gradient(perturbations,child_fitness,bds,mode=mode,config=config,novelties=novelties)
    grad.block_until_ready()
    print(mode," ",time.time()-now)

fitness   0.7055678367614746
evo_var   0.012302637100219727
evo_ent   2.6022915840148926
innovation   0.005745410919189453
quality_evo_var   31.25009036064148
quality_evo_ent   32.8418447971344


KeyboardInterrupt: 

In [3]:
from es_map import nd_sort

In [4]:
multi_objective_fitnesses = np.random.randn(10000,2)
multi_objective_fitnesses_gpu = jnp.array(multi_objective_fitnesses)

In [6]:
%%time
# with new code we do domination matrix calculation on gpu and front calculation with numba
domination_matrix = nd_sort.jax_calculate_domination_matrix(multi_objective_fitnesses_gpu)

# copy back to cpu for rest of the computation
domination_matrix = np.array(domination_matrix) 
multi_objective_fitnesses = np.array(multi_objective_fitnesses)

fronts = nd_sort.numba_calculate_pareto_fronts(domination_matrix)
nondomination_rank_dict = nd_sort.fronts_to_nondomination_rank(fronts)
crowding = nd_sort.calculate_crowding_metrics(multi_objective_fitnesses,fronts)
sorted_indicies = nd_sort.nondominated_sort(nondomination_rank_dict,crowding)

CPU times: user 2.09 s, sys: 63.8 ms, total: 2.16 s
Wall time: 2.14 s


In [23]:
%%time
fronts = nd_sort.calculate_pareto_fronts(multi_objective_fitnesses,use_jax=False)
nondomination_rank_dict = nd_sort.fronts_to_nondomination_rank(fronts)
crowding = nd_sort.calculate_crowding_metrics(multi_objective_fitnesses,fronts)
sorted_indicies = nd_sort.nondominated_sort(nondomination_rank_dict,crowding)

CPU times: user 36 s, sys: 2.35 s, total: 38.4 s
Wall time: 38.4 s


In [26]:
%%time
fronts = nd_sort.calculate_pareto_fronts(multi_objective_fitnesses_gpu,use_jax=True)
nondomination_rank_dict = nd_sort.fronts_to_nondomination_rank(fronts)
crowding = nd_sort.calculate_crowding_metrics(multi_objective_fitnesses,fronts)
sorted_indicies = nd_sort.nondominated_sort(nondomination_rank_dict,crowding)

CPU times: user 29.7 s, sys: 1.15 s, total: 30.8 s
Wall time: 30.8 s


In [30]:
%%time
fronts = nd_sort.calculate_pareto_fronts(multi_objective_fitnesses_gpu,use_jax=True)

CPU times: user 28.6 s, sys: 598 ms, total: 29.2 s
Wall time: 29.2 s


In [28]:
%%time
nondomination_rank_dict = nd_sort.fronts_to_nondomination_rank(fronts)

CPU times: user 1.41 ms, sys: 191 µs, total: 1.6 ms
Wall time: 1.63 ms


In [29]:
%%time
crowding = nd_sort.calculate_crowding_metrics(multi_objective_fitnesses,fronts)

CPU times: user 25.8 ms, sys: 672 µs, total: 26.5 ms
Wall time: 25.6 ms


In [6]:
from numba import jit
@jit(nopython=True)
def calculate_pareto_fronts(domination_matrix,pop_size):
    
    # Calculate dominated set for each individual
    domination_sets = []
    domination_counts = []
    
    for i in range(pop_size):
        current_dimination_set = set()
        domination_counts.append(0)
        for j in range(pop_size):
            if domination_matrix[i,j]:
                current_dimination_set.add(j)
            elif domination_matrix[j,i]:
                domination_counts[-1] += 1
                
        domination_sets.append(current_dimination_set)

    domination_counts = np.array(domination_counts)
    fronts = []
    while True:
        current_front = np.where(domination_counts==0)[0]
        if len(current_front) == 0:
            #print("Done")
            break
        #print("Front: ",current_front)
        fronts.append(current_front)

        for individual in current_front:
            domination_counts[individual] = -1 # this individual is already accounted for, make it -1 so  ==0 will not find it anymore
            dominated_by_current_set = domination_sets[individual]
            for dominated_by_current in dominated_by_current_set:
                domination_counts[dominated_by_current] -= 1
            
    return fronts

@jit(nopython=True)
def calculate_crowding_metrics(fitnesses,fronts):
    
    num_objectives = fitnesses.shape[1]
    num_individuals = fitnesses.shape[0]
    
    # Normalise each objectives, so they are in the range [0,1]
    # This is necessary, so each objective's contribution have the same magnitude to the crowding metric.
    normalized_fitnesses = np.zeros_like(fitnesses)
    for objective_i in range(num_objectives):
        min_val = np.min(fitnesses[:,objective_i])
        max_val = np.max(fitnesses[:,objective_i])
        val_range = max_val - min_val
        val_range = max(val_range,0.001)
        normalized_fitnesses[:,objective_i] = (fitnesses[:,objective_i] - min_val) / val_range
    
    fitnesses = normalized_fitnesses
    crowding_metrics = np.zeros(num_individuals)

    for front in fronts:
        for objective_i in range(num_objectives):
            
            sorted_front = sorted(front,key = lambda x : fitnesses[x,objective_i])
            
            crowding_metrics[sorted_front[0]] = np.inf
            crowding_metrics[sorted_front[-1]] = np.inf
            if len(sorted_front) > 2:
                for i in range(1,len(sorted_front)-1):
                    crowding_metrics[sorted_front[i]] += fitnesses[sorted_front[i+1],objective_i] - fitnesses[sorted_front[i-1],objective_i]

    return  crowding_metrics

In [9]:
fitnesses = np.random.randn(10000,2)
fitnesses_gpu = jnp.array(fitnesses)
domination_matrix = jax_calculate_domination_matrix(fitnesses_gpu).block_until_ready()
domination_matrix = np.array(domination_matrix)

In [12]:
%%time
fronts = calculate_pareto_fronts(domination_matrix,fitnesses.shape[0])

CPU times: user 2.25 s, sys: 0 ns, total: 2.25 s
Wall time: 2.24 s


In [8]:
# Test domination cpu gpu

def calculate_domination_matrix(fitnesses):    
    
    pop_size = fitnesses.shape[0]
    num_objectives = fitnesses.shape[1]
    
    # numpy meshgrid does not work if original array is 2d, so we have to build the mesh grid manually
    fitness_grid_x = np.zeros([pop_size,pop_size,num_objectives])
    fitness_grid_y = np.zeros([pop_size,pop_size,num_objectives])
    
    for i in range(pop_size):
        fitness_grid_x[i,:,:] = fitnesses[i]
        fitness_grid_y[:,i,:] = fitnesses[i]
    
    larger_or_equal = fitness_grid_x >= fitness_grid_y
    larger = fitness_grid_x > fitness_grid_y
    
    return np.logical_and(np.all(larger_or_equal,axis=2),np.any(larger,axis=2))

def calculate_domination_matrix_meshgrid(fitnesses):
    
    pop_size = fitnesses.shape[0]
    num_objectives = fitnesses.shape[1]
    coord_grid_x,coord_grid_y = np.meshgrid(np.arange(pop_size),np.arange(pop_size),indexing="ij")
    
    fitness_grid_x = fitnesses[coord_grid_x]
    fitness_grid_y = fitnesses[coord_grid_y]
    
    larger_or_equal = fitness_grid_x >= fitness_grid_y
    larger = fitness_grid_x > fitness_grid_y
    
    return np.logical_and(np.all(larger_or_equal,axis=2),np.any(larger,axis=2))
    
    
def jax_calculate_domination_matrix(fitnesses):
    
    pop_size = fitnesses.shape[0]
    num_objectives = fitnesses.shape[1]
    coord_grid_x,coord_grid_y = jnp.meshgrid(jnp.arange(pop_size),jnp.arange(pop_size),indexing="ij")
    
    fitness_grid_x = fitnesses[coord_grid_x]
    fitness_grid_y = fitnesses[coord_grid_y]
    
    larger_or_equal = fitness_grid_x >= fitness_grid_y
    larger = fitness_grid_x > fitness_grid_y
    
    return jnp.logical_and(jnp.all(larger_or_equal,axis=2),jnp.any(larger,axis=2))



In [35]:
fitnesses = np.random.randn(10000,2)

In [39]:
%%time
domination_mat = calculate_domination_matrix(fitnesses)

CPU times: user 3.18 s, sys: 124 ms, total: 3.31 s
Wall time: 3.3 s


In [12]:
fitnesses = np.random.randn(10000,2)
fitnesses_gpu = jnp.array(fitnesses)

In [85]:
%%time
a = calculate_domination_matrix(fitnesses)

CPU times: user 6.37 s, sys: 1.12 s, total: 7.48 s
Wall time: 7.48 s


In [17]:
%%time
b = calculate_domination_matrix_meshgrid(fitnesses)

CPU times: user 5.75 s, sys: 1.21 s, total: 6.95 s
Wall time: 6.95 s


In [16]:
%%time
b = jax_calculate_domination_matrix(fitnesses_gpu).block_until_ready()

CPU times: user 34.2 ms, sys: 333 µs, total: 34.5 ms
Wall time: 28.3 ms


In [19]:
%%time
bb = np.array(b)

AttributeError: 'numpy.ndarray' object has no attribute 'block_until_ready'

In [66]:
coord_grid_x,coord_grid_y = np.meshgrid(range(5),range(5))

In [68]:
grid_x = fitnesses[coord_grid_x]

In [70]:
grid_x.shape

(5, 5, 2)

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

(5, 2)

In [106]:
a = np.random.randn(10000,2)

In [107]:
%%time
b = jnp.array(a)

CPU times: user 290 µs, sys: 286 µs, total: 576 µs
Wall time: 572 µs


In [108]:
%%time
c = np.array(b)

CPU times: user 0 ns, sys: 869 µs, total: 869 µs
Wall time: 454 µs
